In [1]:
import pandas as pd
import numpy as np
import math

# Match node puplulation data

In [2]:
column_names = ["From","To","Valid-From","Valid-Until","Days","Departure","Arrival","Flight","Aircraft","Duration"]
data_schedule = pd.read_csv("schedule.txt", sep=";",names=column_names)

#read in the capacity of the aircraft
column_names = ["Name","ICAO","IATA","Capacity","Country"]
data_aircraft = pd.read_csv("aircrafts.txt", sep=";",names=column_names).loc[:, ["Name","ICAO","IATA","Capacity"]]
data_aircraft = data_aircraft.rename(columns={'Name': 'AircraftName'})
data_aircraft = data_aircraft.rename(columns={'IATA': 'Aircraft'})

# read in the airport data
column_names = ["Name","City","Country","IATA","ICAO","Latitude","Longitude","Altitude","Timezone","DST"]
data_airport = pd.read_csv("airports.txt", sep=";",names=column_names).loc[:, ["Name","City","Country","IATA"]]
data_airport = data_airport.rename(columns={'Name': 'Airport'})
# get from airport for merging 
data_airport_from = data_airport.loc[:, ['Airport',"City","Country","IATA"]]
data_airport_from = data_airport_from.rename(columns={'Airport': 'FromAirport','City': 'FromCity','Country': 'FromCountry',"IATA":"From"})
# get to airport for merging 
data_airport_to = data_airport.loc[:, ['Airport',"City","Country","IATA"]]
data_airport_to = data_airport_to.rename(columns={'Airport': 'ToAirport','City': 'ToCity','Country': 'ToCountry',"IATA":"To"})

In [3]:
# repairing the data 

In [4]:
# Loop through the "Valid-From" column of the data_schedule DataFrame
for index, row in enumerate(data_schedule["Valid-From"]):
    # If the value at the current index is '\N'
    if row == '\\N':
        # Set the value at the current index to "2009-01-02"
        data_schedule["Valid-From"][index] = "2009-01-02"

# Loop through the "Valid-Until" column of the data_schedule DataFrame
for index, row in enumerate(data_schedule["Valid-Until"]):
    # If the value at the current index is '\N'
    if row == '\\N':
        # Set the value at the current index to "2009-01-30"
        data_schedule["Valid-Until"][index] = "2009-01-30"

/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [5]:
# 2009-01-05 is Monday, calculate the existing airlines within a week. 2009-01-05 to 2009-01-11

In [6]:
# create a sample dataframe with a date column
df = data_schedule
# convert the date column to datetime
df['Valid-From'] = pd.to_datetime(df['Valid-From'])
# extract the day from the datetime
df['from'] = df['Valid-From'].dt.strftime('%d')
# convert the day to an integer
df['from'] = df['from'].str.extract('(\d+)').astype(int)
df['Valid-Until'] = pd.to_datetime(df['Valid-Until'])
# extract the day from the datetime
df['until'] = df['Valid-Until'].dt.strftime('%d')
# convert the day to an integer
df['until'] = df['until'].str.extract('(\d+)').astype(int)
# filter the dataframe based on the day
#df = df[df['day'] <= 11 and df['day'] >= 5]
df = df.loc[df['until'] >= 12]
data_schedule = df.loc[df['from'] <= 5]

In [7]:
merged_df = pd.merge(data_schedule, data_aircraft, on='Aircraft')
merged_df

,From,To,Valid-From,Valid-Until,Days,Departure,Arrival,Flight,Aircraft,Duration,from,until,AircraftName,ICAO,Capacity
0,AAL,AAR,2009-01-02,2009-01-30,17,19:40,20:05,BA8280,D38,00:25,2,30,Fairchild Dornier Do.328,D328,33
1,AAL,OSL,2009-01-03,2009-01-30,2345,17:25,18:25,BA8275,D38,01:00,3,30,Fairchild Dornier Do.328,D328,33
2,AAL,OSL,2009-01-02,2009-01-30,1234,07:15,08:15,BA8273,D38,01:00,2,30,Fairchild Dornier Do.328,D328,33
3,AAL,OSL,2009-01-02,2009-01-30,17,16:50,17:50,BA8279,D38,01:00,2,30,Fairchild Dornier Do.328,D328,33
4,AAR,OSL,2009-01-03,2009-01-30,2345,16:50,18:00,BA8271,D38,01:10,3,30,Fairchild Dornier Do.328,D328,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9467,NRT,HNL,2009-01-02,2009-01-30,1234567,21:00,08:30,JO74,747,06:30,2,30,Boeing 747 all pax models,\N,416
9468,NRT,KOA,2009-01-03,2009-01-17,1234567,21:00,08:40,JO70,747,06:40,3,17,Boeing 747 all pax models,\N,416
9469,HEL,TMP,2009-01-02,2009-01-30,6,13:40,14:20,AY3997,AT5,00:40,2,30,Aerospatiale/Alenia ATR 42-500,AT45,50
9470,TMP,HEL,2009-01-02,2009-01-30,6,14:40,15:15,AY3998,AT5,00:35,2,30,Aerospatiale/Alenia ATR 42-500,AT45,50


In [8]:
# list_from_name = []
# list_from_City = []
# list_to_name = []
# for index,from_id in enumerate(merged_df["From"].tolist()): "City"
#     list_from_name.append(data_airport.loc[data_airport['IATA'] == from_id, 'Airport'].values[0])
    
# for index,to_id in enumerate(merged_df["To"].tolist()):
#     list_to_name.append(data_airport.loc[data_airport['IATA'] == to_id, 'Airport'].values[0])

In [9]:
merged_df = pd.merge(merged_df, data_airport_from, on="From")
merged_df = pd.merge(merged_df, data_airport_to, on="To")
merged_df = merged_df.drop_duplicates()



In [10]:
# Within one week flights， what's the total number of passengers 

In [11]:
list_passengers = []
for index, (days,capacity) in enumerate(zip(merged_df['Days'],merged_df["Capacity"])):
    #print(len(str(day))*int(capacity))
    list_passengers.append(int(len(str(days))*int(capacity)/7))
merged_df = merged_df.assign(passenger_pre=list_passengers)
merged_df

,From,To,Valid-From,Valid-Until,Days,Departure,Arrival,Flight,Aircraft,Duration,...,AircraftName,ICAO,Capacity,FromAirport,FromCity,FromCountry,ToAirport,ToCity,ToCountry,passenger_pre
0,AAL,AAR,2009-01-02,2009-01-30,17,19:40,20:05,BA8280,D38,00:25,...,Fairchild Dornier Do.328,D328,33,Aalborg,Aalborg,Denmark,Aarhus,Aarhus,Denmark,9
1,OSL,AAR,2009-01-03,2009-01-30,2345,18:25,19:35,BA8272,D38,01:10,...,Fairchild Dornier Do.328,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,18
2,OSL,AAR,2009-01-02,2009-01-30,1234,09:00,10:10,BA8270,D38,01:10,...,Fairchild Dornier Do.328,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,18
3,OSL,AAR,2009-01-02,2009-01-30,17,18:20,20:05,BA8280,D38,01:45,...,Fairchild Dornier Do.328,D328,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,9
4,AAL,OSL,2009-01-03,2009-01-30,2345,17:25,18:25,BA8275,D38,01:00,...,Fairchild Dornier Do.328,D328,33,Aalborg,Aalborg,Denmark,Gardermoen,Oslo,Norway,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9467,SJU,FDF,2009-01-02,2009-01-30,1357,20:20,22:18,AA5020,AT7,01:58,...,Aerospatiale/Alenia ATR 72,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Le Lamentin,Fort-de-france,Martinique,42
9468,SJU,NEV,2009-01-02,2009-01-30,1234567,12:35,13:54,AA5086,AT7,01:19,...,Aerospatiale/Alenia ATR 72,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Vance Winkworth Amory International Airport,\N,Saint Kitts and Nevis,74
9469,SJU,PTP,2009-01-02,2009-01-30,1357,20:30,22:03,AA5012,AT7,01:33,...,Aerospatiale/Alenia ATR 72,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Le Raizet,Pointe-a-pitre,Guadeloupe,42
9470,SJU,SLU,2009-01-02,2009-01-30,1234567,20:15,22:18,AA5018,AT7,02:03,...,Aerospatiale/Alenia ATR 72,AT72,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,George F L Charles,Castries,Saint Lucia,74


In [12]:
# following is the demo for with data in city Aarhus
data_city = pd.read_csv("city_dataset/worldcities.csv").loc[:, ["city","population"]]
#generate the population of to city and from city
data_city = data_city.rename(columns={'city': 'FromCity'})
#data_city
# merged_df = pd.merge(merged_df, data_city, on="FromCity")

In [13]:
data_city

,FromCity,population
0,Tokyo,39105000.0
1,Jakarta,35362000.0
2,Delhi,31870000.0
3,Manila,23971000.0
4,São Paulo,22495000.0
...,...,...
42900,Tukchi,10.0
42901,Numto,10.0
42902,Nord,10.0
42903,Timmiarmiut,10.0


In [14]:
# get the dictionary about city : population 
data_city_list = data_city["FromCity"].tolist()
data_city_population = data_city["population"].tolist()
data_from_list = merged_df["FromCity"].tolist()
data_to_list = merged_df["ToCity"].tolist()
flight_city_list = set(data_from_list+data_to_list)
city_polulation_dict = {}
for city in flight_city_list:
    try:
        popuindex = data_city_list.index(city)
        city_polulation_dict[city] = data_city_population[popuindex]
    except:
        pass
    
#lso get another dictionary for income infected person in cities, initalize it with zero
from_city = merged_df["FromCity"].tolist()
to_city  = merged_df["ToCity"].tolist()
cities_in = list(set(from_city+to_city))
infected_polulation_city = {}
for city_in in cities_in:
    infected_polulation_city[city_in] = 0
    
# Also get another dictionary for income infected person in this term, initalize it with zero
from_airport = merged_df["FromAirport"].tolist()
to_airport = merged_df["ToAirport"].tolist()
airports = list(set(from_airport+to_airport))
infected_polulation_in_airport = {}
for airport in airports:
    infected_polulation_in_airport[airport] = 0
    
    
    
#adding an coreelation airport2city and city2airport：
airport2city = {}
city2airport = {}
for index,(from_airport,from_city,to_airport,to_city) in enumerate(zip(merged_df["FromAirport"].tolist(),merged_df["FromCity"].tolist(),merged_df["ToAirport"].tolist(),merged_df["ToCity"].tolist())):
    airport2city[from_airport] = from_city
    airport2city[to_airport] = to_city
for key, value in airport2city.items():
    city2airport[value] = key

In [15]:
# demo about calculatiing the inital infected person.
# demo for if there is x people infected by virus, which is lambda% of total y people.
# Accordingly, the infected person for this flight is merged_df["passengers"] * lambda%

In [16]:
#if lambda is 3% percentage of people infected in New York, the total infected person is city_polulation_dict["New York"]*0.03
# Then considering all airlines from New York
result_airline = merged_df.loc[merged_df['FromCity'] == "New York"]
# Then we can get  infected person to all destionation  , if infected rate during the travel is 0.4(this need to be calculated by you)
for to_airport, infected_num in zip(result_airline["ToAirport"].tolist(),result_airline["Capacity"].tolist()):
    infected_polulation_in_airport[to_airport]+=int(infected_num)

In [17]:
infected_polulation_in_airport

{'Fort Smith Rgnl': 0,
 'Cotabato': 0,
 'Dallas Love Fld': 0,
 'Talavera La Real': 0,
 'Changle': 0,
 'Lishe': 0,
 'Norfolk Intl': 0,
 'Central Illinois Rgnl': 0,
 'Chacalluta': 0,
 'Tegel': 0,
 'Coronel Francisco Secada Vignetta Intl': 0,
 'Vancouver Intl': 386,
 'Tokyo Intl': 0,
 'Hartsfield Jackson Atlanta Intl': 0,
 'Carthage': 0,
 'Tokunoshima': 0,
 'Incheon Intl': 0,
 'San Angelo Rgnl Mathis Fld': 0,
 'Ontario Intl': 0,
 'Oita': 0,
 'Roma Airport': 0,
 'Hewanorra Intl': 178,
 'Rockhampton': 0,
 'Deputado Luis Eduardo Magalhaes': 0,
 'Genova Sestri': 0,
 'Suvarnabhumi Intl': 0,
 'Stuttgart': 0,
 'Darwin Intl': 0,
 'Miami Intl': 2940,
 'Sacramento Intl': 0,
 'Townsville': 0,
 'Hongqiao Intl': 0,
 'East Texas Rgnl': 0,
 'Shenyang Taoxian International Airport': 0,
 'Pittsburgh Intl': 161,
 'Hong Kong Intl': 1158,
 'Tianhe': 0,
 'Pristina': 0,
 'Phoenix Sky Harbor Intl': 172,
 'Saint Exupery': 0,
 'Blue Grass': 0,
 'General Edward Lawrence Logan Intl': 729,
 'Ronald Reagan Washington

In [18]:
#Accordingly the income infected number of each city is 
for in_airport in infected_polulation_in_airport.keys():
    city = airport2city[in_airport]
    infected_polulation_city[city] += infected_polulation_in_airport[in_airport]
    

In [19]:
infected_polulation_city

{'Tanger': 0,
 'Mendoza': 0,
 'Amman': 303,
 'Indianapolis': 0,
 'El Paso': 0,
 'Windsor Locks': 0,
 'Sacramento': 0,
 'New York': 0,
 'Calcutta': 0,
 'Chiclayo': 0,
 'Aguascalientes': 0,
 'Tokunoshima': 0,
 'Belo Horizonte': 0,
 'HUNTSVILLE': 0,
 'Noumea': 0,
 'Tegucigalpa': 0,
 'Oita': 0,
 'Santa Cruz': 0,
 'Wichita': 0,
 'Rockhampton': 0,
 'Cancun': 356,
 'Wellington': 0,
 'Wassau': 0,
 'Stuttgart': 0,
 'Taramajima': 0,
 '\\N': 0,
 'Townsville': 0,
 'Accra': 0,
 'Grand Junction': 0,
 'Dubuque IA': 0,
 'Toulouse': 0,
 'Pristina': 0,
 'Roadtown/beef Island': 0,
 'The Valley': 0,
 'Baton Rouge': 0,
 'Green Bay': 0,
 'Zagreb': 0,
 "Coff's Harbour": 0,
 'Raleigh-durham': 794,
 'Maracaibo': 0,
 'Houston': 0,
 'Karratha': 0,
 'Dubrovnik': 0,
 'Dallas': 0,
 'Toronto': 1055,
 'Richmond': 0,
 'Barcelona': 269,
 'Alice Springs': 0,
 'Hanamaki': 0,
 'Ushuaia': 0,
 'Oulu': 0,
 'Evansville': 0,
 'Memphis': 0,
 'Colorado Springs': 0,
 'Mount Isa': 0,
 'Punta Arenas': 0,
 'Ninbo': 0,
 'Gibraltar': 

# SEIR model

In [20]:


dup_df = merged_df[['From', 'Departure', 'Flight', 'passenger_pre']]

dup_df = dup_df.duplicated(['From', 'Departure', 'Flight'], keep=False)

merged_df = merged_df.assign(dup_flight=dup_df.to_list())
 
def re_cal_passengers(p, dup):
    if dup:
        return math.ceil(p/2)
    else: 
        return p

merged_df['passenger']= merged_df.apply(lambda a: re_cal_passengers(a.passenger_pre, a.dup_flight), axis=1)
merged_df

,From,To,Valid-From,Valid-Until,Days,Departure,Arrival,Flight,Aircraft,Duration,...,Capacity,FromAirport,FromCity,FromCountry,ToAirport,ToCity,ToCountry,passenger_pre,dup_flight,passenger
0,AAL,AAR,2009-01-02,2009-01-30,17,19:40,20:05,BA8280,D38,00:25,...,33,Aalborg,Aalborg,Denmark,Aarhus,Aarhus,Denmark,9,False,9
1,OSL,AAR,2009-01-03,2009-01-30,2345,18:25,19:35,BA8272,D38,01:10,...,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,18,False,18
2,OSL,AAR,2009-01-02,2009-01-30,1234,09:00,10:10,BA8270,D38,01:10,...,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,18,False,18
3,OSL,AAR,2009-01-02,2009-01-30,17,18:20,20:05,BA8280,D38,01:45,...,33,Gardermoen,Oslo,Norway,Aarhus,Aarhus,Denmark,9,True,5
4,AAL,OSL,2009-01-03,2009-01-30,2345,17:25,18:25,BA8275,D38,01:00,...,33,Aalborg,Aalborg,Denmark,Gardermoen,Oslo,Norway,18,False,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9467,SJU,FDF,2009-01-02,2009-01-30,1357,20:20,22:18,AA5020,AT7,01:58,...,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Le Lamentin,Fort-de-france,Martinique,42,False,42
9468,SJU,NEV,2009-01-02,2009-01-30,1234567,12:35,13:54,AA5086,AT7,01:19,...,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Vance Winkworth Amory International Airport,\N,Saint Kitts and Nevis,74,False,74
9469,SJU,PTP,2009-01-02,2009-01-30,1357,20:30,22:03,AA5012,AT7,01:33,...,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,Le Raizet,Pointe-a-pitre,Guadeloupe,42,False,42
9470,SJU,SLU,2009-01-02,2009-01-30,1234567,20:15,22:18,AA5018,AT7,02:03,...,74,Luis Munoz Marin Intl,San Juan,Puerto Rico,George F L Charles,Castries,Saint Lucia,74,False,74


In [23]:
terminalame = list(adj.columns.values)

# list of names
names = ['Jessa', 'Eric', 'Bob']

with open(r'E:/demos/files_demos/account/sales.txt', 'w') as fp:
    fp.write('\n'.join(names))

['AAL',
 'AAN',
 'AAR',
 'ABI',
 'ABQ',
 'ABV',
 'ABX',
 'ABZ',
 'ACA',
 'ACC',
 'ACE',
 'ACT',
 'ADB',
 'ADE',
 'ADL',
 'AEP',
 'AEX',
 'AGJ',
 'AGP',
 'AGU',
 'AKJ',
 'AKL',
 'ALC',
 'ALG',
 'ALP',
 'ALY',
 'AMA',
 'AMM',
 'AMS',
 'ANF',
 'ANU',
 'AOJ',
 'AQJ',
 'AQP',
 'ARI',
 'ARM',
 'ARN',
 'ASJ',
 'ASP',
 'ATH',
 'ATL',
 'AUA',
 'AUH',
 'AUS',
 'AXA',
 'AXT',
 'AYQ',
 'AYT',
 'AZO',
 'BAH',
 'BBA',
 'BCI',
 'BCN',
 'BDA',
 'BDB',
 'BDL',
 'BEG',
 'BEY',
 'BGI',
 'BHM',
 'BIO',
 'BJX',
 'BJZ',
 'BKI',
 'BKK',
 'BKQ',
 'BLL',
 'BLQ',
 'BLR',
 'BLT',
 'BME',
 'BMI',
 'BNA',
 'BNE',
 'BOD',
 'BOG',
 'BOM',
 'BON',
 'BOS',
 'BRC',
 'BRU',
 'BSL',
 'BTR',
 'BUD',
 'BUF',
 'BUR',
 'BWI',
 'BZE',
 'CAI',
 'CAN',
 'CBR',
 'CCP',
 'CCS',
 'CCU',
 'CDG',
 'CEB',
 'CFE',
 'CFS',
 'CGK',
 'CHA',
 'CHC',
 'CHS',
 'CID',
 'CIX',
 'CJA',
 'CJC',
 'CKG',
 'CLE',
 'CLJ',
 'CLL',
 'CLO',
 'CLT',
 'CMB',
 'CMH',
 'CMI',
 'CMN',
 'CNF',
 'CNS',
 'COR',
 'COS',
 'CPH',
 'CPO',
 'CPT',
 'CQM',
 'CRD',


In [21]:
adj = pd.crosstab(merged_df.From, merged_df.To, values=merged_df.passenger, aggfunc='sum').fillna(0)
adj_matrix = adj.to_numpy()

np.save("init_flight.npy", adj_matrix, allow_pickle=True)